## Exercise 12.1. 

**Question 1:**

To convert data into BERT input format, which function can be used from the BertTokenizer class:

A. encode

B. encode_plus

C. encode_bert

D. A and B

**Answer: D**

**Question 2:**

BERT models are capable of capturing:

A. Text Similarity 

B. Global Context Information

C. Local Context Information

D. All of the above

**Answer: D**


**Question 3:**
The transfomers library from Hugging Face, contain BERT models that cannot be used for:

A. Image Classification

B. Text Classification

C. Both A and B

D. None of the Above


**Answer: A**


## Exercise 12.2

Using the “airline_review.csv” file from "Resources/Datasets", develop a BERT based sequence classifier.


In [1]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

from google.colab import drive
drive.mount('/gdrive')

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

message_dataset= pd.read_csv("/gdrive/My Drive/datasets/ham_spam.csv")

message_dataset.head()
message_dataset.head()  

def clean_text(doc):

    document = remove_tags(doc)

    document = re.sub('[^a-zA-Z]', ' ', document)

    document = re.sub(r"\s+[a-zA-Z]\s+", ' ', document)

    document = re.sub(r'\s+', ' ', document)

    return document

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(document):
    return TAG_RE.sub('', document)

message_dataset["Message"] = message_dataset["Message"].apply(clean_text)
message_dataset["Category"]  = message_dataset["Category"].map({'ham': 1,'spam': 0})

! pip install transformers

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

 
from transformers import (TFBertForSequenceClassification, 
                          BertTokenizer)
 
from tqdm import tqdm

X = message_dataset["Message"].values
y = message_dataset["Category"].values
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 
print("Shape of training data: {0}, \nShape of test data: {1}".format(X_train.shape, X_test.shape))
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
pad_token=0
pad_token_segment_id=0
max_length= 128
 
def text_to_bert_input(reviews):
  input_ids,attention_masks,token_type_ids=[],[],[]
  
  for review in tqdm(reviews,position=0, leave=True):
    bert_inputs = bert_tokenizer.encode_plus(review,add_special_tokens=True, max_length=max_length, truncation = True)
    
    input, token_type = bert_inputs["input_ids"], bert_inputs["token_type_ids"]
    mask = [1] * len(input)
 
    padding_length = max_length - len(input)
 
    input = input + ([pad_token] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type  = token_type  + ([pad_token_segment_id] * padding_length)
    
    input_ids.append(input)
    attention_masks.append(mask)
    token_type_ids.append(token_type)
  
  return [np.asarray(input_ids), 
            np.asarray(attention_masks), 
            np.asarray(token_type_ids)]
X_test_input=text_to_bert_input(X_test)
X_train_input=text_to_bert_input(X_train)
def convert_to_tensors(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y


train_dataset = tf.data.Dataset.from_tensor_slices((X_train_input[0],X_train_input[1],X_train_input[2],y_train)).map(convert_to_tensors).shuffle(100).batch(32)
 
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_input[0],X_test_input[1],X_test_input[2],y_test)).map(convert_to_tensors).batch(64)
model = TFBertForSequenceClassification.from_pretrained("bert-base-cased")


optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
 
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
 
model.summary()
history = model.fit(train_dataset, epochs=1, validation_data=test_dataset)

Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 17.5MB/s 
     |████████████████████████████████| 890kB 29.5MB/s 
     |████████████████████████████████| 1.1MB 40.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=